In [1]:
pwd

'/home/rnshishir/deepmed/CPRD_Cut22/debug'

In [2]:
import pyspark

In [3]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge, cohort_select
from CPRD.functions import merge
from CPRD.functions.modalities import *

from utils.utils import save_obj

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [5]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
# data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'referral': '/home/workspace/datasets/cprd/cprd2021/*/*_Referral_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.

In [7]:
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.config.utils import *
import CPRD.base.table as cprd_table
import pandas as pd
from CPRD.functions.modalities import *

from CPRD.functions.Causal import *
from pyspark.sql import Window
from CPRD.functions.MedicalDictionary import *
from CPRD.config.utils import *
from CPRD.config.spark import read_parquet, read_txtzip, read_txt
from CPRD.functions.cohort_select_causal import *
from CPRD.functions.predictor_extractor import *

In [8]:
dict4recs = load_obj( '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/Documentation/Set 21/linkage_coverage_dictv')

dict4recs


{'hes_apc': ['01/04/1997', '31/10/2020'],
 'hes_acp': ['01/10/1997', '31/03/2008'],
 'hes_ccare': ['01/04/2008', '31/10/2020'],
 'hes_op': ['01/04/2003', '31/10/2020'],
 'hes_ae': ['01/04/2007', '31/03/2020'],
 'hes_did': ['01/04/2012', '31/10/2020'],
 'ons_death': ['02/01/1998', '16/11/2020'],
 'ncras_cr': ['01/01/1990', '31/12/2018'],
 'ncras_sact': ['01/01/2014', '31/12/2018'],
 'ncras_rtds': ['01/04/2012', '31/12/2018']}

# hes outpatient (OP) extraction

In [9]:

def cvt_datestr2time(df, col, year_first=True):
    """
    convert column from string to date type

    format: 1993/01/07
    """
    return F.to_date(df[col] ,"yyyyMMdd")


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
def read_txt2(sc, sqlContext, path):
    """Reads from a TXT file and returns a PySpark DataFrame, dropping malformed rows."""
    
    # Read the file
    file = sc.textFile(path)

    # Extract header
    head = file.first()
    header_cols = head.split("\t")  # Define expected column names
    print(header_cols)
    # Remove header from the content
    content = file.filter(lambda line: line != head).map(lambda k: k.split("\t"))
    
    # Ensure each row has the correct number of columns (DROPMALFORMED logic)
    valid_rows = content.filter(lambda row: len(row) == len(header_cols))
    
    # Convert to Row objects
    row_rdd = valid_rows.map(lambda row: Row(**dict(zip(header_cols, row))))
    
    # Create DataFrame
    df = sqlContext.createDataFrame(row_rdd)
    print(df.columns)
    return df


In [11]:
def read_txt(sc, sqlContext, path):
    """read from txt to pyspark dataframe"""
    file = sc.textFile(path)
    head = file.first()
    content = file.filter(lambda line: line != head).map(lambda k: k.split('\t'))
    df = sqlContext.createDataFrame(content, schema=head.split('\t'))
    return df

### Read all files and join on important columns (patid AND attendkey identifier)

In [12]:
hesopclin = read_txt2(spark.sc, spark.sqlContext, '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hesop_clinical_20_095_DM.txt')
hesfull = read_txt(spark.sc, spark.sqlContext, '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/aurum_hesapc_op_appttable.txt')
hesfull =hesfull .withColumn('eventdate', F.concat(F.col('apptdate').substr(7, 4), F.col('apptdate').substr(4, 2), F.col('apptdate').substr(1, 2)))
hesfull= hesfull.drop('apptdate')
hesfull = hesfull.withColumn('eventdate', cvt_datestr2time(hesfull, 'eventdate'))
hesopclin =hesopclin.join(hesfull, ['patid','attendkey'],'left')
hesopclin

['patid', 'attendkey', 'diag_01', 'diag_02', 'diag_03', 'diag_04', 'diag_05', 'diag_06', 'diag_07', 'diag_08', 'diag_09', 'diag_10', 'diag_11', 'diag_12', 'opertn_01', 'opertn_02', 'opertn_03', 'opertn_04', 'opertn_05', 'opertn_06', 'opertn_07', 'opertn_08', 'opertn_09', 'opertn_10', 'opertn_11', 'opertn_12', 'opertn_13', 'opertn_14', 'opertn_15', 'opertn_16', 'opertn_17', 'opertn_18', 'opertn_19', 'opertn_20', 'opertn_21', 'opertn_22', 'opertn_23', 'opertn_24', 'operstat', 'HES_yr']
['HES_yr', 'attendkey', 'diag_01', 'diag_02', 'diag_03', 'diag_04', 'diag_05', 'diag_06', 'diag_07', 'diag_08', 'diag_09', 'diag_10', 'diag_11', 'diag_12', 'operstat', 'opertn_01', 'opertn_02', 'opertn_03', 'opertn_04', 'opertn_05', 'opertn_06', 'opertn_07', 'opertn_08', 'opertn_09', 'opertn_10', 'opertn_11', 'opertn_12', 'opertn_13', 'opertn_14', 'opertn_15', 'opertn_16', 'opertn_17', 'opertn_18', 'opertn_19', 'opertn_20', 'opertn_21', 'opertn_22', 'opertn_23', 'opertn_24', 'patid']


DataFrame[patid: string, attendkey: string, HES_yr: string, diag_01: string, diag_02: string, diag_03: string, diag_04: string, diag_05: string, diag_06: string, diag_07: string, diag_08: string, diag_09: string, diag_10: string, diag_11: string, diag_12: string, operstat: string, opertn_01: string, opertn_02: string, opertn_03: string, opertn_04: string, opertn_05: string, opertn_06: string, opertn_07: string, opertn_08: string, opertn_09: string, opertn_10: string, opertn_11: string, opertn_12: string, opertn_13: string, opertn_14: string, opertn_15: string, opertn_16: string, opertn_17: string, opertn_18: string, opertn_19: string, opertn_20: string, opertn_21: string, opertn_22: string, opertn_23: string, opertn_24: string, eventdate: date]

### Differentiate procedures and diags in the OP setting

In [14]:
for x in [ 
    'opertn_01',
 'opertn_02',
 'opertn_03',
 'opertn_04',
 'opertn_05',
 'opertn_06',
 'opertn_07',
 'opertn_08',
 'opertn_09',
 'opertn_10',
 'opertn_11',
 'opertn_12',
 'opertn_13',
 'opertn_14',
 'opertn_15',
 'opertn_16',
 'opertn_17',
 'opertn_18',
 'opertn_19',
 'opertn_20',
 'opertn_21',
 'opertn_22',
 'opertn_23',
 'opertn_24']:
    hesopclin = hesopclin.withColumn(x, F.concat(F.lit("PROC_"), F.col(x)))


### Keep only eligible period of "clean" records for HES OP 

In [15]:
hesopclin = hesopclin.withColumn('goodstart', F.to_date(F.lit(DICT2KEEP['hes_op'][0]), 'dd/MM/yyyy')) \
    .withColumn('goodend', F.to_date(F.lit(DICT2KEEP['hes_op'][1]), 'dd/MM/yyyy'))

hesopclin = hesopclin.filter(F.col('eventdate') >= F.col('goodstart')).filter(F.col('eventdate') < F.col('goodend'))
hesopclin =hesopclin.drop('goodend').drop('goodstart')

### Collapse all various diags/operations into a single column - expensive process...

In [16]:
def keep4_rmdot_dash_func(x):
    x= x.replace(".", "")
    x= x.replace("-", "")
    out = x
    if 'PROC_' in x:
        out = out[:9]
    else:
        out = out[:4]
    return out

In [17]:
cause_cols = ['diag_01',
 'diag_02',
 'diag_03',
 'diag_04',
 'diag_05',
 'diag_06',
 'diag_07',
 'diag_08',
 'diag_09',
 'diag_10',
 'diag_11',
 'diag_12',
 'opertn_01',
 'opertn_02',
 'opertn_03',
 'opertn_04',
 'opertn_05',
 'opertn_06',
 'opertn_07',
 'opertn_08',
 'opertn_09',
 'opertn_10',
 'opertn_11',
 'opertn_12',
 'opertn_13',
 'opertn_14',
 'opertn_15',
 'opertn_16',
 'opertn_17',
 'opertn_18',
 'opertn_19',
 'opertn_20',
 'opertn_21',
 'opertn_22', 
 'opertn_23',
 'opertn_24',
             ]

cause_cols = [F.col(each) for each in cause_cols]
hesopclin = hesopclin.withColumn("code", F.array(cause_cols)).select(['patid', 'code', 'eventdate'])

keep4_ = F.udf(lambda x: keep4_rmdot_dash_func(x))

hesopclin = hesopclin.withColumn('code', F.explode('code')) \
    .withColumn('code', keep4_('code')) 

hesopclin = hesopclin.filter(F.col('code')!='PROC_')
hesopclin =  hesopclin.filter((F.col('eventdate') != '') & (F.col('code') != ''))
hesopclin = hesopclin.filter((F.length(F.col("code")) == 9) | (F.length(F.col("code")) == 4))


DataFrame[patid: string, attendkey: string, HES_yr: string, diag_01: string, diag_02: string, diag_03: string, diag_04: string, diag_05: string, diag_06: string, diag_07: string, diag_08: string, diag_09: string, diag_10: string, diag_11: string, diag_12: string, operstat: string, opertn_01: string, opertn_02: string, opertn_03: string, opertn_04: string, opertn_05: string, opertn_06: string, opertn_07: string, opertn_08: string, opertn_09: string, opertn_10: string, opertn_11: string, opertn_12: string, opertn_13: string, opertn_14: string, opertn_15: string, opertn_16: string, opertn_17: string, opertn_18: string, opertn_19: string, opertn_20: string, opertn_21: string, opertn_22: string, opertn_23: string, opertn_24: string, eventdate: date]
DataFrame[patid: string, code: array<string>, eventdate: date]


### Extra filtering by last X in code as often a wildcard character and nonsense procedure codes

In [ ]:
def lastXrem(x):
    out = x
    if x  == x[-1] or 'X' == x[-1]:
        out = out[:-1]
    
    return out
lastX = F.udf(lambda x: lastXrem(x))
hesopclin=hesopclin.withColumn('code', lastX('code')) 
hesopclin =hesopclin.dropDuplicates()
hesopclin =hesopclin.filter(F.col ('code')!= "PROC_XXX")
hesopclin = hesopclin.filter(
    (F.col("code").contains("PROC_X99")) ==False)

In [18]:
# hesopclin.write.parquet('/home/shared/shishir/AurumOut/rawDat/HESOP_DiagProcClean.parquet')


# Quick analysis

In [86]:
hesopclin= read_parquet( spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/HESOP_DiagProcClean.parquet')

In [87]:
# 862,438,748 total records for HES OP
# 24,916,264 unique patient data records in HES OP


### Counts analyses

In [75]:
# hesopclin= read_parquet( spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/HESOP_DiagProcClean.parquet')
# hesopclin = hesopclin.groupBy('code').count()
# hesopclin.write.parquet('/home/shared/shishir/AurumOut/rawDat/HESOP_DiagProcClean_counts.parquet')



In [79]:
hesopclincounts= pd.read_parquet( '/home/shared/shishir/AurumOut/rawDat/HESOP_DiagProcClean_counts.parquet')
hesopclincounts['leng'] = hesopclincounts.code.apply(lambda x: len(x))
hesopclincounts.leng.value_counts()

4    8579
9    8222
3     240
Name: leng, dtype: int64

In [81]:
hesopclincounts.sort_values('count', ascending=False)[:10]

,code,count,leng
5548,R69,641718135,3
7148,PROC_X621,19151211,9
7149,PROC_Y981,11450590,9
6053,PROC_X622,8770521,9
1290,PROC_C873,4505324,9
14862,PROC_U216,3206067,9
9872,PROC_X654,3025998,9
14483,PROC_U263,2895373,9
8599,PROC_Z941,2317505,9
12666,PROC_X623,2316563,9


In [82]:
print('number of proc: ', len(hesopclincounts[hesopclincounts.leng==9]))
print('number of diag: ', len(hesopclincounts[hesopclincounts.leng!=9]))

number of proc:  8222
number of diag:  8819
